In [ ]:
%pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 32kB/s 


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
url='https://raw.githubusercontent.com/projjal1/datasets/master/twitter_unprocessed_sentiment.csv'
file=tf.keras.utils.get_file('twitter_unprocessed_sentiment.csv',url)

81526784/81523245 [==============================] - 0s 0us/step


In [ ]:
df=pd.read_csv(file,encoding='ISO-8859-1')

In [ ]:
df.head()

,review,content
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
df.head()

,review,content
583457,0,@Pizard actually i can't. that's only on the...
369345,0,Boo for being sick and having SAT tomorrow.
613941,0,on my dads mini laptop coz my parents took my ...
993918,4,going to see a man about a #chainsaw. Guess i ...
141808,0,"Boo, I got paid $650ish and down to $300 alrea..."


In [ ]:
#Splitting dataset and creating train and test models 
label,content=df['review'],df['content']

In [ ]:
#Splitting for test and train 
train_label,train_content=label[:750000],content[:750000]
test_label,test_content=label[750000:],content[750000:]

In [ ]:
#Creating class labels for dataset
class_name={0:'negative',2:'neutral',4:'postive'}

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_content.values, train_label.values))

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_content.values, test_label.values))

In [ ]:
for feat, targ in train_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: b"@Pizard actually i can't.   that's only on the new iphone 3GS", Target: 0
Features: b'Boo for being sick and having SAT tomorrow. ', Target: 0
Features: b'on my dads mini laptop coz my parents took my phone off me  so excited for tomorrow night! Yeeey!', Target: 0
Features: b'going to see a man about a #chainsaw. Guess i need now is a leather mask ', Target: 4
Features: b"Boo, I got paid $650ish and down to $300 already  stupid bills. Should be lower but wanted a lil fun money/bike crap (and Kiehl's)", Target: 0


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_dataset.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"@Pizard actually i can't.   that's only on the new iphone 3GS",
       b'Boo for being sick and having SAT tomorrow. ',
       b'on my dads mini laptop coz my parents took my phone off me  so excited for tomorrow night! Yeeey!',
       b'going to see a man about a #chainsaw. Guess i need now is a leather mask ',
       b"Boo, I got paid $650ish and down to $300 already  stupid bills. Should be lower but wanted a lil fun money/bike crap (and Kiehl's)",
       b'@mari_posa2 and last time I talked to natalie, she had a sinus infection!!! ',
       b"@draeme_saekyrFK Oh, very cool. Sounds like something I'd like. ",
       b'bubble bath flowin..  late night soak just needa relax n clear my head... lata gzz',
       b'is fighting flu  Just want to stay in bed.',
       b"just exchanged numbers with wee kee. I'm going to miss cao li and the way she say &quot;PAPA TEA&quot; instead of bubble tea. "],
      dtype=object)>

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 4, 0, 0, 4, 4, 0, 0])>

In [ ]:
import tensorflow_hub as hub

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.19016251, -0.98964953,  1.5253124 ,  1.0483664 , -1.6206441 ,
        -0.98183864, -1.2060972 ,  0.05518226,  0.29017183, -0.07813416,
        -0.83747274,  0.49501196,  0.3512097 ,  0.31149969, -0.2454101 ,
        -0.04993708,  0.53949624, -0.400998  ,  0.07399794, -0.82735187],
       [-0.00445754, -0.53738654, -0.10400346,  0.23639303, -0.34741572,
        -0.757306  , -0.1814598 ,  0.47078183,  0.5264247 ,  0.5911486 ,
        -0.41752082,  0.4177771 , -0.28806335,  0.3477816 , -1.5391304 ,
         0.7494272 ,  0.9905668 , -0.7316129 , -0.37010494,  0.22215508],
       [ 0.82026356, -1.7592599 ,  0.8069903 ,  0.45060226, -1.98169   ,
        -0.41613427, -1.62783   ,  1.4294891 ,  1.0244524 ,  0.70324385,
        -3.9638944 ,  2.0109196 , -0.27472472,  0.22872315, -2.2729747 ,
         0.01729978,  2.2452245 , -1.7211179 , -1.5085258 , -0.4142752 ]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer),
model.add(tf.keras.layers.Dense(16, activation='relu')),
model.add(tf.keras.layers.Dense(16, activation='relu')),
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 400,645
Trainable params: 400,645
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset.shuffle(10000).batch(512), epochs=20)

Epoch 1/20
1465/1465 [==============================] - 10s 7ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 2/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 3/20
1465/1465 [==============================] - 10s 7ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 4/20
1465/1465 [==============================] - 10s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 5/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 6/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 7/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 8/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 9/20
1465/1465 [==============================] - 9s 6ms/step - loss: 1.1311e-07 - accuracy: 0.5769
Epoch 10/20
1465/1465 [====================

In [ ]:
model.evaluate(test_dataset.batch(512), verbose=2)

584/584 - 3s - loss: 1.1287e-07 - accuracy: 0.5774


[1.1287103518498043e-07, 0.5773825645446777]